In [50]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
import pprint
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import ast
import os.path
from datetime import datetime
from collections import Counter
import time
import sys
import re

nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nefarion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/nefarion/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nefarion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
os.chdir("/mnt/f/Linda/Work work/Categorization")

input_file_relative = "Datasets/Dark_web_dataset - Copy (2).csv"
output_file_relative = "Datasets/hopefully_the_end.csv"

char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
top = 1000
MAX_SEQUENCE_LENGTH = top
MAX_NB_WORDS = top
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
stemmer = LancasterStemmer()

all_words = {}

# Read new generated data set file and set en_tokens to ''

In [52]:
df = pd.read_csv(input_file_relative)[['url', 'content', 'category']]
df['tokens_en', 'confidence'] = ''
# Shuffle the rows and reset the index
df = df.sample(frac=1).reset_index(drop=True)
print(f"Loaded {df.shape[0]} rows in {df.shape[1]} columns")

Loaded 4120 rows in 4 columns


# Take only English documents - above a certain threshold

In [53]:
def remove_non_english_documents(data_frame, english_tolerance = 20):
    removed = 0
    english_confidence = []
    tokens_en = []
    for i, document in data_frame.iterrows():
        english_words = 0
        text = document['content']
        
        # Remove long base-64 encoded strings, e.g. images
        text = re.sub("(?:[A-Za-z0-9+/]{4})*(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=|[A-Za-z0-9+/]{4}){24,}","", text)
        
        wordies = nltk.word_tokenize(text)

        tokens = []
        for w in wordies:
            lower = w.lower()
            if lower in english_vocab:
                tokens.append(lower)
                english_words += 1
        tokens_en.append(tokens)
        doc_english_confidence = english_words / len(wordies) * 100
        english_confidence.append(doc_english_confidence)
        if doc_english_confidence <= english_tolerance:
            removed += 1
        
        if i % 100 == 0:
            print("done {}".format(i))
        sys.stdout.flush()
        time.sleep(0.05)
        
    data_frame['english:confidence'] = english_confidence
    data_frame['tokens_en'] = tokens_en
    print(f"Removed {removed} documents considered non-english.")
    return data_frame[data_frame['english:confidence'] > english_tolerance]

In [54]:
df = remove_non_english_documents(df)
#df = pd.read_csv('Datasets/english_tokens.csv')[['url', 'content', 'category', 'tokens_en', 'english:confidence']]

done 0
done 100
done 200
done 300
done 400
done 500
done 600
done 700
done 800
done 900
done 1000
done 1100
done 1200
done 1300
done 1400
done 1500
done 1600
done 1700
done 1800
done 1900
done 2000
done 2100
done 2200
done 2300
done 2400
done 2500
done 2600
done 2700
done 2800
done 2900
done 3000
done 3100
done 3200
done 3300
done 3400
done 3500
done 3600
done 3700
done 3800
done 3900
done 4000
done 4100
Removed 13 documents considered non-english.


# Get labels
# The labels array is a lookup for label (category) names

In [55]:
labels_index = {}  # dictionary mapping label name to numeric id
def get_labels(d_frame):
    data_frame = d_frame.copy(deep = True)
    df_categories = data_frame.drop_duplicates(subset = 'category')
    df_categories = df_categories['category']
    
    index = 0
    for category in df_categories:
        labels_index[category] = index
        index += 1

get_labels(df)
pprint.pprint(labels_index)

{'Art': 6,
 'Deviancy': 2,
 'Drugs': 9,
 'Encyclopedia and Knowledge': 12,
 'Fake Identity and Hitmen': 8,
 'Finance and Fraud': 0,
 'Gambling': 10,
 'Guns': 13,
 'Hosting and Programming and Hacking': 3,
 'Online Marketplace': 11,
 'Other': 1,
 'Porn': 5,
 'Social': 7,
 'Web Catalogue': 4}


# Fill the texts array with the raw content of every page
# Fill the labels_index array with the category id on the index position of the page

In [56]:
texts = []  # list of text samples
labels = []  # list of label ids
def prepare_text_data(data_frame):
    for i, document in data_frame.iterrows():
        text = document['content']
        texts.append(text)
        category = document['category']
        label = labels_index[category]
        labels.append(label)

prepare_text_data(df)

pprint.pprint(texts[:1])    
pprint.pprint(labels[:5])
print(len(texts))
print(len(labels))

['Mr.Millionaire\'s Coaching                           #    # "If somebody '
 'offers you an amazing opportunity but you are not sure you can do it, say '
 'yes - then learn how to do it later!"― Richard Branson  ## Mr.Millionaire '
 'Presents  ## THE BINARY MILLIONAIRE METHOD  For Any Queries, Support or '
 "Questions, Don't Hesitate to Email Me Directly and I will reply within 2 to "
 '24 hours - MrMillionaireSupport@Protonmail.com OR  wickr: mrmillionairex OR '
 'ICQ: 740011109\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b    ## '
 'HOW WE MAKING $77,400--  ## $158,400 a DAY TRADING  ## BINARY OPTIONS IN 15 '
 "MINUTES!!  ## AND HOW YOU CAN DO IT TOO !  ## 100's OF MIND BOGGLING  ## "
 'PROOFS HERE    HELLO HELLO HELLO  YOU FINALLY FOUND US !!!!  WELCOME TO OUR '
 'SECRET WORLD !!!!  THIS IS "THE BINARY MILLIONAIRE METHOD" !!!!!!    __READ '
 'EACH AND EVERY SINGLE WORD ON THIS PAGE TILL__  __THE LAST____ TO KNOW THE '
 'SHOCKING $1,000,000 SECRET CLUB__  __SURPRISE ____AS A

In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [58]:
# first, build index mapping words in the embeddings set
# to their embedding vector
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print(f'Found {len(embeddings_index)} word vectors.')
pprint.pprint(embeddings_index["hello"]) 

Indexing word vectors.
Found 400000 word vectors.
array([ 0.26688  ,  0.39632  ,  0.6169   , -0.77451  , -0.1039   ,
        0.26697  ,  0.2788   ,  0.30992  ,  0.0054685, -0.085256 ,
        0.73602  , -0.098432 ,  0.5479   , -0.030305 ,  0.33479  ,
        0.14094  , -0.0070003,  0.32569  ,  0.22902  ,  0.46557  ,
       -0.19531  ,  0.37491  , -0.7139   , -0.51775  ,  0.77039  ,
        1.0881   , -0.66011  , -0.16234  ,  0.9119   ,  0.21046  ,
        0.047494 ,  1.0019   ,  1.1133   ,  0.70094  , -0.08696  ,
        0.47571  ,  0.1636   , -0.44469  ,  0.4469   , -0.93817  ,
        0.013101 ,  0.085964 , -0.67456  ,  0.49662  , -0.037827 ,
       -0.11038  , -0.28612  ,  0.074606 , -0.31527  , -0.093774 ,
       -0.57069  ,  0.66865  ,  0.45307  , -0.34154  , -0.7166   ,
       -0.75273  ,  0.075212 ,  0.57903  , -0.1191   , -0.11379  ,
       -0.10026  ,  0.71341  , -1.1574   , -0.74026  ,  0.40452  ,
        0.18023  ,  0.21449  ,  0.37638  ,  0.11239  , -0.53639  ,
       -0.02

In [59]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


/home/nefarion/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 129766 unique tokens.
Shape of data tensor: (4107, 1000)
Shape of label tensor: (4107, 14)


In [60]:
print('Preparing embedding matrix.')
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    #else:
    #    print("could not find the word {} in the embeddings dictionary".format(word))

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
use_glove_embeddings = False #it seems that using glove embeddings doesn't bring up any noticeable improvement
if use_glove_embeddings:
    embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
else:
    embedding_layer = Embedding(num_words,EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

Preparing embedding matrix.


In [61]:
print('Training model.')
print(len(labels_index))

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(labels.shape[1], activation='softmax')(x)

model = Model(sequence_input, preds)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

Training model.
14
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 

In [62]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data of {} examples'.format(len(x_val)))
results = model.evaluate(x_val, y_val, batch_size=128)
run_accuracy = results[1]
print('test loss, test acc:', results)

category_id_to_name_lookup = {v: k for k, v in labels_index.items()}

print('\n# Detailed results for training data set')
predictions = model.predict(x_val)
predictions_copy = predictions.copy()
for idx, val in enumerate(predictions):
    category_index = predictions[idx].argmax(axis=0)
    original_category_index = y_val[idx].argmax(axis=0)
    if original_category_index != category_index:
        print("Predicted {} should have been {}"
              .format(category_id_to_name_lookup[category_index], category_id_to_name_lookup[original_category_index]))

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for custom sample')
custom_example = ["Tor Project | Research Tor Project | Research ![](https://research.torproject.org//images/tor- logo.svg)  About Documentation Support Blog Donate  __ Download Tor Browser  ###### Research  ## Home    * Home   * Safety Board   * Research Groups   * Ideas   * Tools   * Tech Reports   * Mailing Lists  Many people around the world are doing research on how to improve the Tor design, what's going on in the Tor network, and more generally on attacks and defenses for anonymous communication systems. This page summarizes the resources we provide to help make your Tor research more effective. You can reach us about research by picking one of the channels listed here.    * **Data**. We've been collecting data to learn more about the Tor network: how many relays and clients there are in the network, what capabilities they have, how fast the network is, how many clients are connecting via bridges, what traffic exits the network, etc. We are also developing tools to process these huge data archives and come up with useful statistics. Let us know what other information you'd like to see, and we can work with you to help make sure it gets collected safely and robustly.   * **Analysis**. If you're investigating Tor, or solving a Tor-related problem, _please_ talk to us somewhere along the way — the earlier the better. These days we review too many conference paper submissions that make bad assumptions and end up solving the wrong problem. Since the Tor protocol and the Tor network are both moving targets, measuring things without understanding what's going on behind the scenes is going to result in bad conclusions. In particular, different groups often unwittingly run a variety of experiments in parallel, and at the same time we're constantly modifying the design to try new approaches. If you let us know what you're doing and what you're trying to learn, we can help you understand what other variables to expect and how to interpret your results.   * **Measurement and attack tools**. We're building a repository of tools that can be used to measure, analyze, or perform attacks on Tor. Many research groups end up needing to do similar measurements (for example, change the Tor design in some way and then see if latency improves), and we hope to help everybody standardize on a few tools and then make them really good. Also, while there are some really neat Tor attacks that people have published about, it's hard to track down a copy of the code they used. Let us know if you have new tools we should list, or improvements to the existing ones. The more the better, at this stage.   * **We need defenses too — not just attacks**. Most researchers find it easy and fun to come up with novel attacks on anonymity systems. We've seen this result lately in terms of improved congestion attacks, attacks based on remotely measuring latency or throughput, and so on. Knowing how things can go wrong is important, and we recognize that the incentives in academia aren't aligned with spending energy on designing defenses, but it sure would be great to get more attention to how to address the attacks. We'd love to help brainstorm about how to make Tor better. As a bonus, your paper might even end up with a stroner section.   * **In-person help**. If you're doing interesting and important Tor research and need help understanding how the Tor network or design works, interpreting your data, crafting your experiments, etc, we can send a Tor researcher to your doorstep. As you might expect, we don't have a lot of free time; but making sure that research is done in a way that's useful to us is really important. So let us know, and we'll work something out.  If you're interested in anonymity research, you must make it to the Privacy Enhancing Technologies Symposium. Everybody who's anybody in the anonymity research world will be there. Stipends are generally available for people whose presence will benefit the community.  To get up to speed on anonymity research, read these papers (especially the ones in boxes). We also keep a list of Tor Tech Reports that are (co-)authored by Tor developers.  Our mission: to advance human rights and freedoms by creating and deploying free and open source anonymity and privacy technologies, supporting their unrestricted availability and use, and furthering their scientific and popular understanding.    * About    * Documentation    * Press    * Blog    * Newsletter    * Contact   Subscribe to our Newsletter  Get monthly updates and opportunities from the Tor Project:  Sign up  Trademark, copyright notices, and rules for use by third parties can be found in our _FAQ_.  "]
custom_sequences = tokenizer.texts_to_sequences(custom_example)
predictions = model.predict(pad_sequences(custom_sequences, maxlen=MAX_SEQUENCE_LENGTH))
category_index = predictions[0].argmax(axis=0)
print(category_id_to_name_lookup[category_index])




# Evaluate on test data of 821 examples
821/821 [==============================] - 2s 3ms/step
test loss, test acc: [0.8573498459448913, 0.8574908375740051]

# Detailed results for training data set
Predicted Other should have been Hosting and Programming and Hacking
Predicted Drugs should have been Encyclopedia and Knowledge
Predicted Porn should have been Deviancy
Predicted Hosting and Programming and Hacking should have been Porn
Predicted Hosting and Programming and Hacking should have been Other
Predicted Web Catalogue should have been Other
Predicted Hosting and Programming and Hacking should have been Guns
Predicted Finance and Fraud should have been Online Marketplace
Predicted Hosting and Programming and Hacking should have been Other
Predicted Finance and Fraud should have been Online Marketplace
Predicted Deviancy should have been Porn
Predicted Deviancy should have been Other
Predicted Porn should have been Web Catalogue
Predicted Other should have been Hosting and Program

In [63]:
print('\n# Dump tokenizer so it can be used for tokenizing on unseen data with the same word dictionary.')
import pickle
with open('Models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print("\n# Saving model to disk")
model_json = model.to_json()
with open("Models/DarkWebCategoryModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Models/DarkWebCategoryModel.h5")


# Dump tokenizer so it can be used for tokenizing on unseen data with the same word dictionary.

# Saving model to disk


In [64]:
import time
start = time.time()

print('\n# Test loaded model and tokenizer')

print('\n# Loading Tokenizer')
import pickle
with open('Models/tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
    
print('\n# Loading saved Model')
from keras.models import model_from_json
json_file = open('Models/DarkWebCategoryModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
print('\n# Loading saved Model Weights')
loaded_model.load_weights("Models/DarkWebCategoryModel.h5")
print('\n# Compiling loaded Model')
loaded_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print("Loading tokenizer, model, weights and compiling it took {} seconds.".format(time.time() - start))
 
print('\n# Use loaded model on a custom exmaple')
custom_example = ["Tor Project | Research Tor Project | Research ![](https://research.torproject.org//images/tor- logo.svg)  About Documentation Support Blog Donate  __ Download Tor Browser  ###### Research  ## Home    * Home   * Safety Board   * Research Groups   * Ideas   * Tools   * Tech Reports   * Mailing Lists  Many people around the world are doing research on how to improve the Tor design, what's going on in the Tor network, and more generally on attacks and defenses for anonymous communication systems. This page summarizes the resources we provide to help make your Tor research more effective. You can reach us about research by picking one of the channels listed here.    * **Data**. We've been collecting data to learn more about the Tor network: how many relays and clients there are in the network, what capabilities they have, how fast the network is, how many clients are connecting via bridges, what traffic exits the network, etc. We are also developing tools to process these huge data archives and come up with useful statistics. Let us know what other information you'd like to see, and we can work with you to help make sure it gets collected safely and robustly.   * **Analysis**. If you're investigating Tor, or solving a Tor-related problem, _please_ talk to us somewhere along the way — the earlier the better. These days we review too many conference paper submissions that make bad assumptions and end up solving the wrong problem. Since the Tor protocol and the Tor network are both moving targets, measuring things without understanding what's going on behind the scenes is going to result in bad conclusions. In particular, different groups often unwittingly run a variety of experiments in parallel, and at the same time we're constantly modifying the design to try new approaches. If you let us know what you're doing and what you're trying to learn, we can help you understand what other variables to expect and how to interpret your results.   * **Measurement and attack tools**. We're building a repository of tools that can be used to measure, analyze, or perform attacks on Tor. Many research groups end up needing to do similar measurements (for example, change the Tor design in some way and then see if latency improves), and we hope to help everybody standardize on a few tools and then make them really good. Also, while there are some really neat Tor attacks that people have published about, it's hard to track down a copy of the code they used. Let us know if you have new tools we should list, or improvements to the existing ones. The more the better, at this stage.   * **We need defenses too — not just attacks**. Most researchers find it easy and fun to come up with novel attacks on anonymity systems. We've seen this result lately in terms of improved congestion attacks, attacks based on remotely measuring latency or throughput, and so on. Knowing how things can go wrong is important, and we recognize that the incentives in academia aren't aligned with spending energy on designing defenses, but it sure would be great to get more attention to how to address the attacks. We'd love to help brainstorm about how to make Tor better. As a bonus, your paper might even end up with a stroner section.   * **In-person help**. If you're doing interesting and important Tor research and need help understanding how the Tor network or design works, interpreting your data, crafting your experiments, etc, we can send a Tor researcher to your doorstep. As you might expect, we don't have a lot of free time; but making sure that research is done in a way that's useful to us is really important. So let us know, and we'll work something out.  If you're interested in anonymity research, you must make it to the Privacy Enhancing Technologies Symposium. Everybody who's anybody in the anonymity research world will be there. Stipends are generally available for people whose presence will benefit the community.  To get up to speed on anonymity research, read these papers (especially the ones in boxes). We also keep a list of Tor Tech Reports that are (co-)authored by Tor developers.  Our mission: to advance human rights and freedoms by creating and deploying free and open source anonymity and privacy technologies, supporting their unrestricted availability and use, and furthering their scientific and popular understanding.    * About    * Documentation    * Press    * Blog    * Newsletter    * Contact   Subscribe to our Newsletter  Get monthly updates and opportunities from the Tor Project:  Sign up  Trademark, copyright notices, and rules for use by third parties can be found in our _FAQ_.  "]
custom_sequences = loaded_tokenizer.texts_to_sequences(custom_example)
predictions = loaded_model.predict(pad_sequences(custom_sequences, maxlen=MAX_SEQUENCE_LENGTH))
category_index = predictions[0].argmax(axis=0)
print(category_id_to_name_lookup[category_index])


# Test loaded model and tokenizer

# Loading Tokenizer

# Loading saved Model

# Loading saved Model Weights

# Compiling loaded Model
Loading tokenizer, model, weights and compiling it took 0.7262263298034668 seconds.

# Use loaded model on a custom exmaple
Hosting and Programming and Hacking


In [70]:
sorted_labels = [cat_key for cat_key in labels_index]
sorted_labels.sort()
# Initiate confusion matrix
matrix = {cat_key: {} for cat_key in sorted_labels}
for row in matrix:
    matrix[row] = {cat_key: 0 for cat_key in sorted_labels}

len_of_longest = 0
for label in sorted_labels:
    label_len = len(label)
    len_of_longest = label_len if label_len > len_of_longest else len_of_longest
       
# Create confusion matrix
for idx, val in enumerate(predictions_copy):
    category_index = predictions_copy[idx].argmax(axis=0)
    original_category_index = y_val[idx].argmax(axis=0)
    category = category_id_to_name_lookup[category_index]
    original_category = category_id_to_name_lookup[original_category_index]
    matrix[original_category][category] += 1

# Simplify confusion matrix
def get_percentage(res, values_sum):
    return f'{res/values_sum*100:.2f}' if values_sum else '0.00'

simple_matrix = matrix.copy()
for row in simple_matrix:
    values = simple_matrix[row].values()
    values_sum = sum(values)
    results_row = [ '{:11}'.format(f'{res}: {get_percentage(res, values_sum)}%') for res in list(values) ]
    beau_results = [res.replace('"', '') for res in results_row]
    simple_matrix[row] = beau_results
        
# Print confusion matrix
print(f'accuracy: {run_accuracy}')
short_labels = ['OnlnMarket', 'WebCat', 'Porn', 'FinFraud', 'Other', 'Social', 'HacProgr', 'Encyklo', 'Deviancy', 'Drugs', 'FakeIdnHitm', 'Guns', 'Gambling', 'Art']
short_labels.sort()
formatted_short_labels = ['{:11}'.format(lab) for lab in short_labels]
print("{:{len_of_longest}}: {}".format(' ', formatted_short_labels, len_of_longest=len_of_longest))
for row in simple_matrix:
    print("\n{:{len_of_longest}}: {}".format(row, simple_matrix[row], len_of_longest=len_of_longest))


accuracy: 0.8574908375740051
                                   : ['Art        ', 'Deviancy   ', 'Drugs      ', 'Encyklo    ', 'FakeIdnHitm', 'FinFraud   ', 'Gambling   ', 'Guns       ', 'HacProgr   ', 'OnlnMarket ', 'Other      ', 'Porn       ', 'Social     ', 'WebCat     ']

Art                                : ['0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '1: 100.00% ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ']

Deviancy                           : ['0: 0.00%   ', '17: 70.83% ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '0: 0.00%   ', '1: 4.17%   ', '0: 0.00%   ', '0: 0.00%   ', '6: 25.00%  ', '0: 0.00%   ', '0: 0.00%   ']

Drugs                              : ['0: 0.00%   ', '0: 0.00%   ', '10: 76.92% ', '0: 0.00%   ', '0: 0.00%   ', '1: 7.69%   ', '0: 0.00%   ', '0: 0.00%   ', '1: 7.69%   ', '0: 0.00%   ', '1: 7.69%   ', '0: 0.00%   ', '0: 0.